# LibraryData

## Objectives:
### Convert timestamps to minutes per hour
### Display computer id with total minutes used per hour, total log-ons in same hour

The following attempts to take non-uniform timestamps and do the following:
* resample those timestamps into minutes
* forward-fill the states
* resample the minutes into hours, summing the minutes

The function `.resample()` changed in pandas 0.18.1, so this was a learning process.

In [1]:
import numpy as np
import pandas as pd
print("This should be '0.20.1':")
print("pandas:         " + str(pd.__version__))
print("This should be '1.12.1':")
print("numpy:          " + str(np.__version__))

This should be '0.20.1':
pandas:         0.20.1
This should be '1.12.1':
numpy:          1.12.1


importing the data from the Library data sample.

In [2]:
useData = pd.read_csv(r'../data/170830_StateData.csv',parse_dates=[3])

Verifying the status of the columns. 'datastamp' should be a datetime64 field.

In [3]:
useData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3873 entries, 0 to 3872
Data columns (total 4 columns):
machineName    3873 non-null object
location       3873 non-null object
state          3873 non-null object
datestamp      3873 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 121.1+ KB


Testing functionality with a single computer. Working with iterating across the numpy array later.

In [4]:
computerDataName = 'CRR005'

In [5]:
computerTimeArray = useData[useData.machineName == computerDataName]

In [6]:
computerTimeArray

,machineName,location,state,datestamp
0,CRR005,Current Periodicals,in-use,2017-08-31 09:09:00
4,CRR005,Current Periodicals,restarted,2017-08-31 09:07:00
6,CRR005,Current Periodicals,offline,2017-08-31 09:07:00
7,CRR005,Current Periodicals,available,2017-08-31 09:07:00
64,CRR005,Current Periodicals,in-use,2017-08-31 08:45:00
248,CRR005,Current Periodicals,restarted,2017-08-31 06:30:00
461,CRR005,Current Periodicals,available,2017-08-30 22:39:00
655,CRR005,Current Periodicals,in-use,2017-08-30 20:18:00
687,CRR005,Current Periodicals,available,2017-08-30 19:57:00
784,CRR005,Current Periodicals,in-use,2017-08-30 19:17:00


This turns the 'in-use' value into true, and all of the others into false. Since this analysis is based upon when a machine is not being used (as opposed to when it is offline/available/restarted) all other states are irrelevant.

Will need to investigate copy/view on this error. May need to do this as a dataframe with `.concat()`.

In [7]:
computerTimeArray.loc[:,'state'] = pd.Series(computerTimeArray.state == 'in-use')

C:\Users\Patricia\Anaconda2\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Location data is irrelevant for a machine at this point. Also, location can be derived from machine name, machine location is (at this point) not that precise.

Pandas was giving a duplicate error due to the three non- 'in-use' states occurring simultaneously. Dropping unused columns for data duplication (machinename and location), and dropping timestamp duplications.

In [8]:
computerTimeArray = computerTimeArray.loc[:,'state':'datestamp'].drop_duplicates()

In [9]:
computerTimeArray = computerTimeArray.set_index('datestamp').sort_index()

In [10]:
computerTimeArray

,state
datestamp,
2017-08-30 06:30:00,False
2017-08-30 09:23:00,True
2017-08-30 09:43:00,False
2017-08-30 10:03:00,True
2017-08-30 11:18:00,False
2017-08-30 11:23:00,False
2017-08-30 11:25:00,True
2017-08-30 11:44:00,False
2017-08-30 11:47:00,True


This takes the above data and resamples it into minute increments. Value for the specific minute is put into place, while 'NaN' values will take on the previous non-NaN data.

In [11]:
computerTimeArrayMin = computerTimeArray.resample('T').ffill()

In [12]:
computerTimeArrayPerHour = computerTimeArrayMin.resample('H').sum()

In [13]:
computerTimeArrayPerHour

,state
datestamp,
2017-08-30 06:00:00,0.0
2017-08-30 07:00:00,0.0
2017-08-30 08:00:00,0.0
2017-08-30 09:00:00,20.0
2017-08-30 10:00:00,57.0
2017-08-30 11:00:00,50.0
2017-08-30 12:00:00,60.0
2017-08-30 13:00:00,46.0
2017-08-30 14:00:00,60.0


## Alternative method of computing per-machine utilization

In [14]:
# Compute the intervals during which each computer was in use.
# @df must be a pandas DataFrame with columns ['machineName',
# 'location', 'state', 'datestamp'].
# Returns a dictionary keyed by 'machineName' with values
# of (timestamp, timerange) corresponding to sessions of machine
# usage.
def toUsageIntervals(df):
    # Bin records by @machineName.
    binned = {}
    names = df['machineName'].unique()
    for name in names:
        matching = df[df['machineName'] == name]
        binned[name] = matching.loc[:,'state':'datestamp'].sort_values(by='datestamp')

    # Compute date ranges of usage for each machine.
    for (name, frame) in binned.items():
        # Mark which records correspond to 'in-use'.
        inUse = list(frame['state'] == 'in-use')
        # Each consecutive True/False corresponds to a period of activity.
        # Compute a DateRange from each consecutive True/False pair.
        indexedInUse = list(zip(range(0,len(inUse),1), inUse))
        transitions = [(frame.iloc[x[0]].loc['datestamp'], frame.iloc[y[0]].loc['datestamp']) for (x, y) in zip(indexedInUse[:-1], indexedInUse[1:]) if x[1]==True and y[1]==False]
        dateRanges = [(x, y-x) for (x, y) in transitions]
        binned[name] = dateRanges
    return binned

from pandas.tseries.offsets import *
import functools
import itertools

# Convert date ranges into hourly usage data.
def usageIntervalsToPercentUtilization(binned, period='h'):
    def dateRangeToPeriodUtilization(begin, length):
        # Round the starting time down to the nearest period.
        start = begin.floor(period)
        if ((begin + length).floor(period) == start):
            # If we don't cross a period boundary, just return the amount of time
            # spent 'in-use' for this period.
            return [(start, pd.Timedelta(length))]
        else:
            # Otherwise, we need to break the session down by period, according to wall-time.
            # We already have the beginning of this interval.  Round up the end time to the
            # nearest period to get the end-point of the interval.
            end = (begin + length).ceil(period)
            # Now create a list of timestamps corresponding to periods on the clock for this
            # session.
            # One would think 'closed=None' means don't include the first or last value
            # in the interval.  This doesn't seem to be the case though, so manually remove
            # those values.  Otherwise, we end up with two copies of the start period, and
            # two copies of the end period.
            periods = pd.date_range(start, end, freq=period, closed=None)[1:-1]
            # Compute the amount of time spent 'in-use' in the first hour-long interval.
            first = pd.Timedelta('1' + period) - (begin - start)
            # Similarly for the last hour-long interval.
            last = pd.Timedelta('1' + period) - (end - (begin + length))
            # Create a list of (timestamp,timerange) pairs giving the total amount of time
            # spent 'in-use' for each period-long interval.
            return list(zip([start] + list(periods), [first] + ([pd.Timedelta('1' + period)] * (len(periods) - 1)) + [last]))

    def concat(lists):
        return [] if len(lists)==0 else functools.reduce(lambda x, y: x + y, lists)

    def dateRangeToDataFrame(dateRanges):
        # Convert each date range into a list of (timestamp,timerange) where each timestamp
        # corresponds to a period within the timerange.
        next_ = concat(list(map(lambda x: dateRangeToPeriodUtilization(x[0], x[1]), dateRanges)))
        # Group and sum times by period to remove duplicate periods, which would happen if
        # we have multiple timeranges within a period, e.g. if we have two entries
        # ('2017-08-31 09:02:00', '+00:05:00') and ('2017-08-31 09:10:00', '+00:13:00').
        dic = {}
        for (x, y) in next_:
            if x not in dic:
                dic[x] = y
            else:
                dic[x] += y
        # Group the results into a pair of ([timestamp],[timerange]) with unique timestamps.
        z = list(zip(*dic.items()))
        # return a value of type [(timestamp, double)]
        return [(x, y / pd.Timedelta(1, unit=period)) for (x, y) in dic.items()]

    # utilization has type [(name, [(timestamp, double)])]
    utilization = [(x, dateRangeToDataFrame(y)) for (x, y) in binned.items()]
    # get the unique list of timestamps in the data set
    all_timestamps = map(lambda x: map(lambda y: y[0], x[1]), utilization)
    unique_timestamps = functools.reduce(lambda x, y: set(list(x) + list(y)), all_timestamps, set([]))
    sorted_timestamps = sorted(unique_timestamps)
    # sorted_timestamps has type [timestamp]
    # add a bogus (timestamp, 0.0) value for each unique timestamp to the data for each computer
    bogus = [(x, 0.0) for x in sorted_timestamps]
    bogus_utilization = [(x, y + bogus) for (x, y) in utilization]
    # now, per computer, group data by timestamp.  each group will contain at most one valid (i.e. measured)
    # data point, and one bogus data point.
    groups = [(x, itertools.groupby(sorted(y, key=lambda z: z[0]), key=lambda z: z[0])) for (x, y) in bogus_utilization]
    # groups has type [(name, [(timestamp, [(timestamp, double)])])]
    # for each computer, select the entry in each group representing the higher utilization.  the only
    # options within each group are actual reported utilization, or zero.  so if the machine was used,
    # the number we're looking for is the only non-zero value.
    full_utilization = [(x, [sorted(w, key=lambda v: v[1], reverse=True)[0][1] for (_, w) in group]) for (x, group) in groups]
    return pd.DataFrame(dict(full_utilization), index=sorted_timestamps)

utilization = usageIntervalsToPercentUtilization(toUsageIntervals(useData))
utilization

In [15]:
crr005 = utilization['CRR005']
crr005.where(lambda x: x>0).dropna()

In [ ]:
computerTimeArrayPerHour.where(lambda x: x>0).dropna()

CRR005
INC001
CRR006
RIS01
TC209
JLPL0028
INC004
INC015
RIS20
MLC0006
CRR003
CRR004
INC007
MLC0005
INC003
CRR031
RIS003
CRR024
INC006
MLC0002
INC013
DMC0022
CRR009
RIS010
RIS24
RIS016
CRR012
INC009
TC206
BL001
DMC0006
DMC0010
INC010
MLC0015
MLC0013
MLC0014
TC701
MLC0001
RIS027
CRR017
RIS23
MLC0003
INC005
RIS004
CRR019
CRR015
RIS18
TC30001
RIS09
MLC0004
INC008
CRR002
CRR011
CRR029
JLPL0008
RIS006
DMC0002
DMC026
JLPL041
TC212
MLC0007
TC211
CRR039
INC014
DMC0014
TC20001
INC012
DMC0032
TL30002
RRK001
DMC0009
DMC027
RIS035
CRR013
DMC025
DMC029
TL702
TL901
CRR028
RIS029
RIS030
MLC0009
CRR014
RIS02
CRR016
DMC0011
TC8001
CRR010
CRR037
TL6002
DMC030
CRR018
DMC0034
TL5002
CRR020
DMC0001
DMC0018
TC901
CRR021
DMC0017
CITI018
INC011
RIS14
DMC0013
MLC0011
DMC0020
DMC0012
RIS26
RIS21
DMC0033
DMC0015
DMC0019
DMC0016
MLC0008
TC205
TC208
CRR001
CRR027
CRR034
MLC0016
RIS13
RIS12
DMC0023
CRR032
RIS15
CRR023
MLC0010
MLC0012
DMC0021
MLC0017
DMC0031
DMC0007
DMC0024
DMC0008
DMC0005
DDL0001
TC204
MLC0018
CRR03